# <font color='blue'>Data Science Academy - Machine Learning</font>

# <font color='blue'>Desafio DSA - Solução</font>

Generalizando Modelos Lineares:

http://scikit-learn.org/stable/modules/linear_model.html

O objetivo de qualquer modelo de Machine Learning é generalizar para novoso conjuntos de dados. Não criamos o modelo para os dados de treino e sim para novos dados. Por isso é importante encontrar os melhores parâmertros para o modelo, que permitirão generalizar para novos dados. Existem diversas técnicas de regularização, de acordo com o modelo. Abaixo as 3 principais técnicas de regularização para modelos lineares: Ridge, LASSO e ElasticNet.

In [3]:
# Import
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score, KFold, StratifiedKFold
from sklearn.metrics import make_scorer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings("ignore")

/home/andrioli/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
# Carga de dados e definição de variáveis  
boston = load_boston() 
dataset = pd.DataFrame(boston.data, columns = boston.feature_names)
dataset['target'] = boston.target

In [5]:
# Datasets
observations = len(dataset)
variables = dataset.columns[:-1]

In [8]:
# Variáveis X e Y
X = dataset.ix[:,:-1]
y = dataset['target'].values

In [9]:
# Dados de treino e de teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 101)

In [11]:
print ("Tamanho do Dataset de Treino: %i" % len(X_train))
print ("Tamanho do Dataset de Teste: %i" % len(X_test))

Tamanho do Dataset de Treino: 354
Tamanho do Dataset de Teste: 152


In [14]:
# Amostras de Treinom, Validação e Teste
X_train, X_out_sample, y_train, y_out_sample = train_test_split(X, y, test_size = 0.40, random_state = 101)
X_validation, X_test, y_validation, y_test = train_test_split(X_out_sample, y_out_sample, test_size = 0.50, random_state = 101)

In [15]:
print ("Tamamnho da Amostra do Dataset de Treino: %i" % len(X_train))
print ("Tamamnho da Amostra do Dataset de Validação: %i" % len(X_validation))
print ("Tamamnho da Amostra do Dataset de Teste: %i" % len(X_test))

Tamamnho da Amostra do Dataset de Treino: 303
Tamamnho da Amostra do Dataset de Validação: 101
Tamamnho da Amostra do Dataset de Teste: 102


In [16]:
# Calcula o RMSE
def RMSE(y_true, y_pred):
    return np.sum((y_true - y_pred)**2)

In [17]:
# Cria o regressor linear
lm = LinearRegression()

In [18]:
# Divide as amostras em grupos
cv_iterator = KFold(n = len(X), n_folds = 10, shuffle = True, random_state = 101)

In [19]:
# Define os limites do histograma
edges = np.histogram(y, bins = 5)[1]
edges

array([ 5., 14., 23., 32., 41., 50.])

In [20]:
# Retorna os índices dos compartimentos (bins) aos quais pertence cada valor na matriz de entrada.
binning = np.digitize(y, edges)
binning

array([3, 2, 4, 4, 4, 3, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 1, 2,
       2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 1, 1, 1, 2, 2, 2, 3, 3, 4, 3, 3, 3,
       2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 4, 3, 3, 3, 2, 2, 2, 2, 3, 4, 3,
       2, 2, 2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2, 2, 3, 3, 3, 2, 3, 3, 2, 2,
       3, 3, 2, 2, 2, 3, 2, 3, 2, 4, 5, 4, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       3, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 1, 5, 3, 3, 3, 6, 6, 6, 2, 3, 6, 3, 3, 2, 2, 2, 3, 3, 2, 3,
       3, 3, 3, 4, 4, 4, 4, 4, 3, 3, 6, 4, 3, 4, 4, 3, 4, 3, 3, 6, 4, 3,
       4, 4, 4, 3, 5, 5, 6, 2, 3, 2, 3, 2, 2, 2, 2, 3, 3, 3, 3, 3, 2, 3,
       3, 2, 3, 3, 5, 6, 4, 3, 5, 3, 3, 3, 5, 5, 3, 3, 3, 3, 3, 3, 2, 2,
       2, 3, 2, 2, 3, 2, 3, 3, 3, 3, 3, 5, 2, 2, 5, 6, 4, 3, 4, 5, 5, 3,
       4, 2, 3, 6, 5, 2, 2, 3, 3, 4, 4, 4, 4, 4, 3, 4, 5, 4, 5, 6, 4, 2,
       2, 3, 2, 3, 3, 4, 3, 3, 2, 3, 3, 2, 2, 3, 3,

In [21]:
# Divide as amostras em grupos com a mesma percentagem
stratified_cv_iterator = StratifiedKFold(binning, n_folds = 10, shuffle = True, random_state = 101)

Um padrão comum no aprendizado da máquina é usar modelos lineares treinados em funções não-lineares dos dados. Esta abordagem mantém o desempenho geralmente rápido dos métodos lineares, enquanto permite que eles se ajustem a uma gama muito maior de dados. Por exemplo, uma regressão linear simples pode ser estendida construindo características polinomiais a partir dos coeficientes. 

In [22]:
# Combinação polinomial de todos os atributos
# Gera uma nova matriz de características constituída por todas as combinações polinomiais 
# das características com grau menor ou igual ao grau especificado.
second_order = PolynomialFeatures(degree = 2, interaction_only = False)
second_order

PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)

In [23]:
type(second_order)

sklearn.preprocessing.data.PolynomialFeatures

In [24]:
# Combinação polinomial de todos os atributos
third_order = PolynomialFeatures(degree = 3, interaction_only = True)
third_order

PolynomialFeatures(degree=3, include_bias=True, interaction_only=True)

In [25]:
# Aplicando a combinação polinominal grau 2 aos atributos de entrada
over_param_X = second_order.fit_transform(X)
over_param_X

array([[1.00000000e+00, 6.32000000e-03, 1.80000000e+01, ...,
        1.57529610e+05, 1.97656200e+03, 2.48004000e+01],
       [1.00000000e+00, 2.73100000e-02, 0.00000000e+00, ...,
        1.57529610e+05, 3.62766600e+03, 8.35396000e+01],
       [1.00000000e+00, 2.72900000e-02, 0.00000000e+00, ...,
        1.54315409e+05, 1.58310490e+03, 1.62409000e+01],
       ...,
       [1.00000000e+00, 6.07600000e-02, 0.00000000e+00, ...,
        1.57529610e+05, 2.23851600e+03, 3.18096000e+01],
       [1.00000000e+00, 1.09590000e-01, 0.00000000e+00, ...,
        1.54802902e+05, 2.54955600e+03, 4.19904000e+01],
       [1.00000000e+00, 4.74100000e-02, 0.00000000e+00, ...,
        1.57529610e+05, 3.12757200e+03, 6.20944000e+01]])

In [26]:
# Aplicando a combinação polinominal grau 3 aos atributos de entrada
extra_over_param_X = third_order.fit_transform(X)
extra_over_param_X

array([[1.00000000e+00, 6.32000000e-03, 1.80000000e+01, ...,
        2.25534240e+04, 5.85062352e+05, 3.02413986e+04],
       [1.00000000e+00, 2.73100000e-02, 0.00000000e+00, ...,
        3.93714640e+04, 8.77895172e+05, 6.45724548e+04],
       [1.00000000e+00, 2.72900000e-02, 0.00000000e+00, ...,
        1.73596280e+04, 3.83111386e+05, 2.81792672e+04],
       ...,
       [1.00000000e+00, 6.07600000e-02, 0.00000000e+00, ...,
        3.23341200e+04, 6.11114868e+05, 4.70088360e+04],
       [1.00000000e+00, 1.09590000e-01, 0.00000000e+00, ...,
        3.71498400e+04, 6.96028788e+05, 5.35406760e+04],
       [1.00000000e+00, 4.74100000e-02, 0.00000000e+00, ...,
        4.51760400e+04, 8.53827156e+05, 6.56790120e+04]])

In [28]:
# Gerando o Score. Variáveis de entrada (X) com pré-processamento polinomial
cv_score = cross_val_score(lm, over_param_X, y, cv = cv_iterator, scoring = 'mean_squared_error', n_jobs = 1)

In [29]:
print (cv_score)

[-12.80006826 -20.16124409  -8.77421975 -19.68028923 -11.66012303
  -7.67918749 -12.82851349 -18.18670763 -35.08897199 -14.23988323]


In [22]:
print ('Cv score: mean %0.3f std %0.3f' % (np.mean(np.abs(cv_score)), np.std(cv_score)))

Cv score: mean 16.156 std 7.464


In [23]:
cv_score = cross_val_score(lm, over_param_X, y, cv = stratified_cv_iterator, scoring = 'neg_mean_squared_error', n_jobs = 1)

In [24]:
print ('Cv score: mean %0.3f std %0.3f' % (np.mean(np.abs(cv_score)), np.std(cv_score)))

Cv score: mean 15.985 std 7.347


Métricas de Avaliação

http://scikit-learn.org/stable/modules/model_evaluation.html

## Regularização

### Ridge

http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html

In [25]:
from sklearn.linear_model import Ridge

In [26]:
# Cria o regressor Ridge
ridge = Ridge(normalize = True)

In [27]:
# Aplica o fit (Ridge) às variáveis
ridge.fit(second_order.fit_transform(X), y)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=True, random_state=None, solver='auto', tol=0.001)

In [28]:
# Aplica o fit (Linear Regression) às variáveis
lm.fit(second_order.fit_transform(X), y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [29]:
# Comparação entre os coeficentes não regularizados (Linear Regression) e regularizados (Ridge)
# O objetivo da regularização é generalizar o modelo para novos conjuntos de dados
print ('Média dos Coeficientes: Não-regularizado = %0.3f Ridge = %0.3f' % (np.mean(lm.coef_), np.mean(ridge.coef_)))
print ('Coeficiente Mínimo: Não-regularizado = %0.3f Ridge = %0.3f' % (np.min(lm.coef_), np.min(ridge.coef_)))
print ('Max coefficient: Não-regularizado = %0.3f Ridge = %0.3f' % (np.max(lm.coef_), np.max(ridge.coef_)))

Média dos Coeficientes: Não-regularizado = 7419953.550 Ridge = -0.027
Coeficiente Mínimo: Não-regularizado = -38.968 Ridge = -2.013
Max coefficient: Não-regularizado = 779094976.171 Ridge = 1.181


### Lasso

http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html

In [30]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.grid_search import RandomizedSearchCV
from scipy.stats import expon

In [31]:
# Cria o regressor LASSO
lasso_reg = Lasso(alpha = 1.0, normalize = True, max_iter = 2*10**5)

In [32]:
# Random seed para garantir a reproducibilidade
np.random.seed(101)

In [33]:
# Divide as amostras em grupos com a mesma percentagem
stratified_cv_iterator = StratifiedKFold(binning, n_folds = 10, shuffle = True, random_state = 101)

In [34]:
# Busca pela melhor combinação de parâmetros testando diversos valores de alfa: {'alpha':np.logspace(-5,2,100)}
# A função RandomizedSearchCV gera o modelo como resultado
func_busca = RandomizedSearchCV(estimator = lasso_reg, 
                                param_distributions = {'alpha':np.logspace(-5,2,100)}, 
                                n_iter = 10, 
                                scoring='mean_squared_error', 
                                n_jobs = 1, 
                                iid = False, 
                                refit = True, 
                                cv = stratified_cv_iterator)

In [35]:
# Aplica o modelo criado aos datasets
# Esta operação leva algum tempo para ser executada.....
func_busca.fit(second_order.fit_transform(X), y)

RandomizedSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[3 2 ... 2 1], n_folds=10, shuffle=True, random_state=101),
          error_score='raise',
          estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=200000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
          fit_params={}, iid=False, n_iter=10, n_jobs=1,
          param_distributions={'alpha': array([1.00000e-05, 1.17681e-05, ..., 8.49753e+01, 1.00000e+02])},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring='mean_squared_error', verbose=0)

In [36]:
# Imprime os resultados
print ('Melhor valor de alpha: %0.5f' % func_busca.best_params_['alpha'])
print ('Melhor CV mean squared error: %0.3f' % np.abs(func_busca.best_score_))

Melhor valor de alpha: 0.00014
Melhor CV mean squared error: 12.185


In [37]:
print ('Coeficients com valor igual a zero: %i de %i' % (np.sum(~(func_busca.best_estimator_.coef_ == 0.0)), 
                                                         len(func_busca.best_estimator_.coef_)))

Coeficients com valor igual a zero: 76 de 105


In [38]:
# Aplica a regularização Lasso com Cross Validation (o melhor modelo é selecionado por Cross Validation)
lasso_reg_cv = LassoCV(alphas = np.logspace(-5,2,100), normalize = True, n_jobs = 1, cv = None, max_iter = 10**6)

In [39]:
# Fit
# Esta operação leva algum tempo para ser executada.....
lasso_reg_cv.fit(second_order.fit_transform(X), y)

LassoCV(alphas=array([1.00000e-05, 1.17681e-05, ..., 8.49753e+01, 1.00000e+02]),
    copy_X=True, cv=None, eps=0.001, fit_intercept=True, max_iter=1000000,
    n_alphas=100, n_jobs=1, normalize=True, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
    verbose=False)

In [40]:
# Imprime os resultados
print ('Melhor valor de alpha: %0.5f' % lasso_reg_cv.alpha_)

Melhor valor de alpha: 0.01097


### Elasticnet

http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html

In [41]:
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn.grid_search import RandomizedSearchCV
from scipy.stats import expon

In [42]:
# Cria o regressor ElasrticNet
elasticnet_reg = ElasticNet(alpha = 1.0, l1_ratio = 0.15, normalize = True, max_iter = 10**6, random_state = 101)

In [43]:
np.random.seed(101)

In [44]:
# Busca pela melhor combinação de parâmetros testando diversos valores de alfa: {'alpha':np.logspace(-5,2,100)}
# A função RandomizedSearchCV gera o modelo como resultado
func_busca = RandomizedSearchCV(estimator = elasticnet_reg, 
                                param_distributions = {'alpha':np.logspace(-5,2,100), 
                                                       'l1_ratio':np.arange(0.0, 1.01, 0.05)}, 
                                n_iter = 10, 
                                scoring = 'mean_squared_error', 
                                n_jobs = 1, 
                                iid = False, 
                                refit = True, 
                                cv = stratified_cv_iterator)

In [45]:
# Fit
func_busca.fit(second_order.fit_transform(X), y)

RandomizedSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[3 2 ... 2 1], n_folds=10, shuffle=True, random_state=101),
          error_score='raise',
          estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.15,
      max_iter=1000000, normalize=True, positive=False, precompute=False,
      random_state=101, selection='cyclic', tol=0.0001, warm_start=False),
          fit_params={}, iid=False, n_iter=10, n_jobs=1,
          param_distributions={'alpha': array([1.00000e-05, 1.17681e-05, ..., 8.49753e+01, 1.00000e+02]), 'l1_ratio': array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ])},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring='mean_squared_error', verbose=0)

In [46]:
# Resultados
print ('Melhor valor de alpha: %0.5f' % func_busca.best_params_['alpha'])
print ('Melhor L1_ratio: %0.5f' % func_busca.best_params_['l1_ratio'])
print ('Melhor CV mean squared error: %0.3f' % np.abs(func_busca.best_score_))

Melhor valor de alpha: 0.00002
Melhor L1_ratio: 0.60000
Melhor CV mean squared error: 11.900


In [47]:
# Coeficientes 
print ('Coeficients com valor igual a zero: %i de %i' % (np.sum(~(func_busca.best_estimator_.coef_ == 0.0)), 
                                                         len(func_busca.best_estimator_.coef_)))

Coeficients com valor igual a zero: 102 de 105


In [48]:
# Aplica a regularização ElasticNet com Cross Validation (o melhor modelo é selecionado por Cross Validation)
elasticnet_reg_cv = ElasticNetCV(alphas = np.logspace(-5,2,100), 
                                 normalize = True, 
                                 n_jobs = 1, 
                                 cv = None, 
                                 max_iter = 10**6)

In [49]:
# Fit
elasticnet_reg_cv.fit(second_order.fit_transform(X), y)

ElasticNetCV(alphas=array([1.00000e-05, 1.17681e-05, ..., 8.49753e+01, 1.00000e+02]),
       copy_X=True, cv=None, eps=0.001, fit_intercept=True, l1_ratio=0.5,
       max_iter=1000000, n_alphas=100, n_jobs=1, normalize=True,
       positive=False, precompute='auto', random_state=None,
       selection='cyclic', tol=0.0001, verbose=0)

In [50]:
# Resultados
print ('Melhor valor de alpha: %0.5f' % elasticnet_reg_cv.alpha_)
print ('Melhor L1_ratio: %0.5f' % elasticnet_reg_cv.l1_ratio_)

Melhor valor de alpha: 0.01292
Melhor L1_ratio: 0.50000


# Fim

### Obrigado - Data Science Academy - <a href="http://facebook.com/dsacademybr">facebook.com/dsacademybr</a>